# Scenario Querying


Here is where I actually to put this data into practice. In this notebook I use my database to compile a playlist of songs appropriate for a "DJ gig scenario" — for example using ratings such  as engagement and happiness to pick out an appropriate collection of songs for a Thursday night wine bar gig.

In this notebook I query my database to output a collection of songs that meet a series of mood/style score thresholds. 

In [1]:
#Imports
import sqlite3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100)
plt.style.use("ggplot")

In [4]:
#Imports from project package called project_tools
from project_tools.utils import adapt_array, convert_array, tag_cleaner, json_opener, table_loader

In [3]:
#Register these functions with sqlite3 so that we I can work with 
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

#Connect to db
conn = sqlite3.connect("../jaage.db", detect_types= sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

Load in tags, genres, and mood scores data.

In [5]:
tags = table_loader(conn, "SELECT * FROM TAGS", apply_function=None)
genres = table_loader(conn, "SELECT * FROM effnet_genres", apply_function=lambda x:x[0].mean())

keep_gcols = np.load("../keep_genre_cols.pkl", allow_pickle=True).tolist()
genres = genres[keep_gcols]

In [7]:
mood_queries = ["SELECT sid, danceable FROM danceability_effnet_discogs_1_activations",
          "SELECT sid, approachable FROM approachability_2c_effnet_discogs_1_activations",
          "SELECT sid, engaging FROM engagement_2c_effnet_discogs_1_activations",
          "SELECT sid, acoustic FROM mood_acoustic_effnet_discogs_1_activations",
          "SELECT sid, aggressive FROM mood_aggressive_effnet_discogs_1_activations",
          "SELECT sid, happy FROM mood_happy_effnet_discogs_1_activations",
          "SELECT sid, sad FROM mood_sad_effnet_discogs_1_activations",
          "SELECT sid, party FROM mood_party_effnet_discogs_1_activations",
            "SELECT sid, bright FROM timbre_effnet_discogs_1_activations"]

mood_scores = table_loader(conn,*mood_queries, apply_function=lambda x:x[0].mean())
mood_scores.head()

,danceable,approachable,engaging,acoustic,aggressive,happy,sad,party,bright
sid,,,,,,,,,
b806881a54bdbf9dd93a290716adf191,0.996905,0.610395,0.962023,0.016339,0.046733,0.323608,0.068570,0.859001,0.470228
46e54d2ab920a088b77382e04877141b,0.974400,0.159482,0.993588,0.033767,0.431848,0.179142,0.035047,0.964722,0.412579
a204ddef5763df6d8f7677701fe9d96f,0.996295,0.382782,0.970832,0.005479,0.079633,0.297905,0.038722,0.972226,0.459330
960097894e83c5810a9c649f17a4e551,0.999633,0.859654,0.996604,0.008332,0.019154,0.825074,0.040252,0.915114,0.461304
a3c1f277aa0110ffc418bf5fa3aa16aa,0.835657,0.888687,0.879341,0.310990,0.006924,0.596215,0.399294,0.308618,0.497131


## Scenarios

There are three main scenarios I want to analyze here.

**Nighttime Wine Bar:** A lowkey and chill evening that's absent of dancing. In this scenario the music I play is apart of the background as opposed to taking center stage and thus the music should reflect that. No aggressive or highly energetic tracks. Mix in some eclectic and leftfield material as well.


**Club:** Obviously a very danceable and energetic. However I'll be varying things but trying to curate based on whether or not I'm an opener or headliner. I'll also incorporate genre as well to see what songs I should play for a disco vs a house party.


**Day Party:** This scenario is meant represent songs radiate jovial and upbeat vibes. I'm aiming for a mix of danceable and non-danceable songs. I basically want to curate the perfect soundtrack for sipping on margheritas in the sun.

### Wine Bar

**Criteria:**

    - Low danceability
    - Low aggressiveness
    - Low party
    - High approachability
    - High engagement
    - Moderate acousticness

First let's grab threshold scores based on the criteria by using percentiles.

In [9]:
class SongQuery:
    
    def __init__(self, name):
        self.queries = []
        self.name = name
        self.df = None
        self.n_songs = None
        
    def add_threshold_query(self, style, percentile, comparison):
        
        threshold = mood_scores[style].quantile(q = percentile).round(3)  
        single_que = "{} {} {}".format(style, comparison, threshold)
        self.queries.append(single_que)
    
    def query_songs(self):
        
        query = " & ".join(self.queries)
        filtered_mood_scores = mood_scores.query(query)
        self.df = pd.concat([tags[["title", "artist"]], filtered_mood_scores], axis = 1 , join = "inner")
        self.n_songs = self.df.shape[0]
    
    def __str__(self):
        
        return f"Style Thresholds for {self.name} scenario:\n\n" + " \n".join(self.queries)

In [10]:
winebar_query = SongQuery("wine bar")
winebar_query.add_threshold_query("danceable", .2, "<=")
winebar_query.add_threshold_query("aggressive", .3, "<=")
winebar_query.add_threshold_query("party", .3, "<=")
winebar_query.add_threshold_query("approachable", .7, ">=")
winebar_query.add_threshold_query("engaging", .7, ">=")
winebar_query.add_threshold_query("acoustic", .5, ">=")
winebar_query.query_songs()
print(winebar_query)
print("\nThere are {} number of songs".format(winebar_query.n_songs))

Style Thresholds for wine bar scenario:

danceable <= 0.978 
aggressive <= 0.031 
party <= 0.888 
approachable >= 0.848 
engaging >= 0.986 
acoustic >= 0.013

There are 17 number of songs


We get a total of 17 songs satisfying our criteria. TBH this number is lower than expected and not even half the size of a playlist for a two-hour gig.

In [11]:
winebar_df = winebar_query.df
winebar_df[["title", "artist"]]

,title,artist
sid,,
df75484fe690b6d47c3fa0a9189d4e20,Baume - love Edit #8 - 03 Cloudburst - I'm Loving You (Baume love Edit),None
bdff71b40ebf9620ce209ee98b18e8ce,Julien Clerc - Coeur de Rocker (Réédit by franck Cassy) 116BPM,CassyTribute
db2a69963b867d9a2e55c9e00b60ccf5,Ndola-ngo,Tim and Foty
190a813d592143fc5b923f9c330ea998,Bernardo Pinheiro - Quero pouco quero muito (Bernardo Pinheiro Edit)_PN,None
019889202700f9696305d7c4ee2189e0,A1 Shams Dinn - Hedi Bled Noum,Smiling C
ef20c3f26b08ac309f04ee0dd4ef8a2d,4-Fire_Flight_-_Wantin'_U_PN,None
87557838f29e5ddf5526de866aef7446,Colourful Environment,Gboyega Adelaja
c1becbc465751149b42ad8ebbe06d80c,"Şehrazat - Kendim ettim, kendim buldum (disco funk, Turkey 1981)",ultradiskopanorama
d048a50d988f683b61d16cd3dc46d8c7,Bian Kou,NST Cophie's


After my assessment, I can confidently say that I would select most of these songs for this scenario. The exception being "Disco Luismi" a remix of 80s Mexican hit pop song "Ahora Te Puedes Marchar" which is too upbeat for a wine bar.

This playlist has a good mix of eclectic groove and languid vibes that would pair well with imbibing a pinot.

Some of my favorite tracks:

- [Bian Kou - NST Cophie's](https://www.youtube.com/watch?v=p2Ih8abpajs&ab_channel=KalitaRecords)

- [I Like It - Maxwell Udoh](https://www.youtube.com/watch?v=2L5Y0drzrqY&ab_channel=Andreapassenger)

- [Ndola-ngo Tim & Foty](https://www.youtube.com/watch?v=g9DITj9yFR4&ab_channel=Tim%26Foty-Topic)

The relatively few number of songs makes me want to see how songs it outputs by relaxing the criteria. Let's see how many we get from eliminating the acoustic threshold and increasing the party threshold from the 30th percentile to 50.

In [12]:
winebar_query2 = SongQuery("wine bar")
winebar_query2.add_threshold_query("danceable", .2, "<=")
winebar_query2.add_threshold_query("aggressive", .3, "<=")
winebar_query2.add_threshold_query("party", .5, "<=")
winebar_query2.add_threshold_query("approachable", .7, ">=")
winebar_query2.add_threshold_query("engaging", .7, ">=")
winebar_query2.query_songs()
print("\nThere are {} number of songs".format(winebar_query2.n_songs))


There are 32 number of songs


### Club

**Criteria:**

- High danceability
- High party 
- Somewhat high aggressiveness
- High engagement

In [13]:
club_query = SongQuery("Club")
club_query.add_threshold_query("danceable", .75, ">=")
club_query.add_threshold_query("party", .75, ">=")
club_query.add_threshold_query("engaging", .75, ">=")
club_query.add_threshold_query("aggressive", .5, ">=")
club_query.query_songs()
print(club_query)
print("\nThere are {} number of songs".format(club_query.n_songs))

Style Thresholds for Club scenario:

danceable >= 0.998 
party >= 0.968 
engaging >= 0.988 
aggressive >= 0.052

There are 169 number of songs


My database holds 169 songs that score in the top quarter percentile for danceability, party, and engagement and in the top half of aggression.

169 songs is definitely too many songs for a playlist, even for a 4-hour gig. However it certainly doesn't hurt to cast a wide net using my database. Once I have this pool of songs, I can then go through it and manually filter them.

Here's are random sample of 10 songs.

In [14]:
club_query.df.sample(10)

,title,artist,approachable,danceable,engaging,acoustic,aggressive,happy,sad,party
sid,,,,,,,,,,
a962a979c673c4d6f8362e655c705792,Flash in the Night (Italo Disco),Public Passion,0.932098,0.998504,0.997739,0.001591,0.071478,0.934145,0.022980,0.989092
faae16e41133a9d4f9755b558c0ec280,Fammi Un Po... (edit),Paul Older,0.869448,0.999726,0.991617,0.002435,0.052591,0.685965,0.029756,0.974431
bc2ab307b0f9404719e45114ebe77183,Genius Of Love (Yungness & Jaminn EDIT),Tom Tom Club,0.334708,0.998044,0.991411,0.004856,0.264875,0.446491,0.025667,0.985328
a235bd476023681e5e4e4c9ad3a65ff8,Give Me - DJ BlakStone Stonewashed Edit,I Level,0.828773,0.999263,0.990798,0.004781,0.061514,0.669370,0.044882,0.974769
91293fe85b47033334dadf2c34700588,Bizarre_Inc-Playing_With_Knives-Quadrant_Mix-27556345_PN,None,0.182421,0.999728,0.993082,0.001753,0.411094,0.385339,0.024521,0.989023
7862eb2ef3a5412652a4d8705ab47a88,Voguing (Edit),Tucan Discos,0.305648,0.999806,0.996138,0.001257,0.232184,0.760545,0.012264,0.990959
20dcf2357346266d51b85913d6cc927c,A1. FrescoEdits - Everybody,The Fresco,0.590066,0.999145,0.991822,0.001264,0.083040,0.386393,0.014648,0.985269
2e0b01dbaa441a676714139d850df743,Old School (Club Edit),DJ Tonka,0.434804,0.999839,0.997454,0.002222,0.273484,0.672146,0.023612,0.994325
4cd485aa5cbbf6001abb4d1084e4cd14,Modern Talking - Brother Louie '98 (Manu Maelso Edit 2022) 118 Bpm,Dj Manu Maelso,0.911369,0.999631,0.994526,0.001928,0.255676,0.907142,0.017561,0.985564


From a cursory inspection of the 169 songs, I can say the criteria I used did a suitable job in picking out "club" songs. 

However just becasue these 169 songs can be played in a club does not mean they all can be played in the same club.

There is a quite a lot of variance in the genres and musical styles of these songs. That's why I want to incorporate the genre scores to produce genre-specific versions of the data.

#### Club Scenario: Disco Edition

Let's use the disco scores from the `genres` dataframe to filter out songs that meet a certain disco threshold score.

In [15]:
disco_scores = genres.electronic___disco.apply(lambda x:x.mean())

disco_threshold = disco_scores.quantile(q=.75)


top_disco_scores = disco_scores[disco_scores >= disco_threshold]

disco_df = club_query.df.join(top_disco_scores, how = "inner")
disco_df.shape

(54, 11)

Applying the 75th percentile disco score threshold to the club dataframe cuts it down from 169 to 54 songs. A setlist size in the mid-50s is pretty typical for a 2-3 hour gig.

In [16]:
disco_df.sample(5)

,title,artist,approachable,danceable,engaging,acoustic,aggressive,happy,sad,party,electronic___disco
sid,,,,,,,,,,,
cff38c09ee24656e75b73d2fd13268e7,Let's Go_PN,DiscoGram,0.884973,0.999710,0.994624,0.003962,0.055196,0.764256,0.062700,0.971718,0.306172
38b291e07e543d467e85f8b3df08cc31,Afro Funky Nuggets,Sun Rhythms,0.511623,0.999831,0.991067,0.002351,0.132539,0.600069,0.028600,0.985297,0.343458
bdaf99e981e8a7f9590443cfbe77f26c,Lady - Easy Love (Club Mix - DJ Rick Mitchell Edit Video),DJ Rick Mitchell Video Edits,0.909428,0.999956,0.995289,0.002772,0.076493,0.861730,0.031057,0.989899,0.491448
32c1ddfe1e22ea50a15cfd66ecbf1cdb,Eko,DiscoGram,0.757704,0.999836,0.993348,0.003457,0.085512,0.718756,0.025944,0.982758,0.355768
53b0ea339f5820fd59cce1c387273394,C. Da Afro - Pink Footpath (Edit),C. Da Afro,0.844733,0.999214,0.993513,0.002082,0.057516,0.563493,0.034084,0.974258,0.548429


Some of my favorite tracks:

- [Let's Go - DiscoGram](https://soundcloud.com/disco-gram/discogram-lets-go)

- [Afro Funky Nuggets - Sun Rhythms](https://www.youtube.com/watch?v=fDgvBKUqhCE&ab_channel=MaslowUnknown)

- [Fammi Un Po - Paul Older](https://www.youtube.com/watch?v=7hXwW3DDPQI&ab_channel=Gazzz696)

#### Club Scenario: House Edition

In [17]:
house_scores = genres.electronic___house.apply(lambda x:x.mean())

house_threshold = house_scores.quantile(q=.75)

top_house_scores = house_scores[house_scores >= house_threshold]

house_df = club_query.df.join(top_house_scores, how = "inner")
print(f"There are {house_df.shape[0]} songs")

There are 66 songs


In [18]:
house_df.sample(5)

,title,artist,approachable,danceable,engaging,acoustic,aggressive,happy,sad,party,electronic___house
sid,,,,,,,,,,,
9c4385c5a60717acb009a44f913cfcdb,Blow Your Mind,BDK,0.226946,0.999957,0.993482,0.001885,0.161408,0.439313,0.024808,0.997239,0.582501
44f4e135f4704c0ddc0892f1c5ab83f5,Is It All Over My Face (Original Mix),CASSIMM,0.499398,0.999717,0.989015,0.006590,0.151336,0.464069,0.034798,0.980033,0.522459
454a12d524a214ac407101b3a67e48e1,Brighter Days (Marco Lys Remix)_PN,"Cajmere, Dajae",0.183513,0.999548,0.993254,0.010616,0.085148,0.246215,0.051595,0.969106,0.484078
63c8af6e5f39ba6e3a67387fe6c7a113,Good Time (Pure),Sound Factory,0.686431,0.999787,0.994815,0.002524,0.153263,0.689357,0.027992,0.995182,0.720590
4808c30d34a5e8c535c20dcb83780d59,26-Ezirk_-_Our_Love_Story_PN,None,0.797058,0.999403,0.996083,0.002970,0.070548,0.825678,0.025402,0.979291,0.733015


Some of my favorite tracks:

- [Searching For Love (Alex Virgo Remix) - Loftus III](https://www.youtube.com/watch?v=dvX5CMOh4Yk&ab_channel=030esar0303)

- [Cassius 99 (Remix)	Cassius](https://www.youtube.com/watch?v=wMTt13lbBQE&ab_channel=sneldersgloves)

- [Ao Ao Ao - Kellit](https://www.youtube.com/watch?v=cNc8n5ihVZs&ab_channel=FreeHouse)

#### Club Scenario: Opening/earlier act

DJing in a club doesn't always mean spinning pulsating bangers. If you're the opening act your job is to basically preheat the crowd for the subsequent acts, so if you throw the extremly danceable and energetic stuff you might run afoul of the other DJs and the promoter.

So for this scenario I want to curate danceable songs that aren't too energetic and aggressive, something to get the audience swaying and bumping but not bouncing all over the place.

**Criteria:**

- Somewhat to highly danceable
- Somewhat to highly party
- At least somewhat engaging
- Neither aggressive nor sad

In [19]:
club_query = SongQuery("Opening Act Club")
club_query.add_threshold_query("danceable", .5, ">=")
club_query.add_threshold_query("danceable", .9, "<=")
club_query.add_threshold_query("party", .5, ">=")
club_query.add_threshold_query("party", .9, "<=")
club_query.add_threshold_query("aggressive", .5, "<=")
club_query.add_threshold_query("engaging", .5, ">=")
club_query.add_threshold_query("sad", .5, "<=")
club_query.query_songs()
print(club_query)
print("\nThere are {} number of songs".format(club_query.n_songs))

Style Thresholds for Opening Act Club scenario:

danceable >= 0.994 
danceable <= 1.0 
party >= 0.936 
party <= 0.983 
aggressive <= 0.052 
engaging >= 0.977 
sad <= 0.059

There are 174 number of songs


In [20]:
club_query.df.sample(5)

,title,artist,approachable,danceable,engaging,acoustic,aggressive,happy,sad,party
sid,,,,,,,,,,
9b162c8ea4b71ce20a823d61cd3865fc,DiscoGram - DG001 - 02 DiscoGram - Soiree,None,0.849071,0.999904,0.991197,0.002912,0.032254,0.829511,0.047949,0.975744
099d4638d5109a055b986de9cc05bee9,Midnight Runners - Tagalog - 01 Control Me_PN,None,0.867271,0.998299,0.987633,0.005121,0.029563,0.741982,0.053618,0.962195
97d7e0177a5d82223525ba7c5e6c9a56,Shit Hot Soundsystem - Good Girl (Shit Hot Soundsystem Edit)_PN,None,0.853661,0.994724,0.984077,0.007173,0.033124,0.619718,0.036373,0.982861
9196fbb58f5446219e7a1ac522b3382d,Shiba_Bam_PN,TOMAS MALO/DJEKO & K'YOU,0.826331,0.995824,0.977856,0.010933,0.032934,0.547223,0.044473,0.959621
1aed7682298a015d3d11a5cefaddb2fe,01 Say What U Wanna Say_PN,None,0.571760,0.999171,0.982709,0.004825,0.021444,0.616826,0.042632,0.952178


Some of my favorite tracks:

- [Something More (Hot Toddy Disco Redux) - Crazy P](https://soundcloud.com/data-transmission/crazy-p-something-more-hot-toddy-disco-redux1)

- [Money - Jack Tennis](https://soundcloud.com/jacktennis/money)

- [Lovin (Soulful Re-edit)](https://soundcloud.com/ff_edits/ff-edits-lovin-soulful-re-edit)

### Day Party

**Criteria**

- Highly happy
- Highly engaging
- Highly approachable
- At least somewhat danceable
- Not aggressive nor sad

In [21]:
day_party = SongQuery("Day Party")
day_party.add_threshold_query("danceable", .5, ">=")
day_party.add_threshold_query("happy", .75, ">=")
day_party.add_threshold_query("approachable", .75, ">=")
day_party.add_threshold_query("engaging", .75, ">=")
day_party.add_threshold_query("aggressive", .5, "<=")
day_party.add_threshold_query("sad", .5, "<=")
day_party.query_songs()
print(day_party)
print("\nThere are {} number of songs".format(day_party.n_songs))

Style Thresholds for Day Party scenario:

danceable >= 0.994 
happy >= 0.737 
approachable >= 0.865 
engaging >= 0.988 
aggressive <= 0.052 
sad <= 0.059

There are 59 number of songs


In [22]:
day_party.df.sample(5)

,title,artist,approachable,danceable,engaging,acoustic,aggressive,happy,sad,party
sid,,,,,,,,,,
9e1fea2e77568e65eee38313dcc87293,Man Problems,The Patchouli Brothers,0.899576,0.999874,0.996975,0.003187,0.021991,0.814914,0.032447,0.985603
297a8b8f471264e63e8a67274e100782,Dance Motif - Stages (Dance Motif Edit),None,0.945070,0.999867,0.997956,0.000911,0.009404,0.968240,0.019091,0.987358
98730bb03937084a79230c6b8d6fddce,Gli Occhi del Mio Ex (Tamati Remix),Auroro Borealo feat. Ariele Frizzante,0.936470,0.997597,0.988971,0.005683,0.026758,0.811369,0.027119,0.983799
89708e55f570612ddf661696e4e1d6a5,Pasteur Lappe - Na Real Sekele Fo' Ya (Diamond Setter Edit),Diamond Setter,0.888393,0.996987,0.996247,0.004826,0.049422,0.876502,0.026907,0.987881
8972f79eec1a64dc11250768c8367ff8,Candi Staton - When You Wake Up (Aaron Swales Edit),"Aaron Swales, Billy Idle, Crystal Touch, Even Funkier",0.927379,0.999778,0.992659,0.006683,0.013066,0.742117,0.053800,0.954653


Some of my favorite tracks:

- [Sugar My Love - T.Z. Junior](https://www.youtube.com/watch?v=CbzkH81E_X8&ab_channel=JAMWAX)

- [Saffa Saphela - Band of Misfits](https://www.youtube.com/watch?v=qnX4bzANvFM&ab_channel=SoundStationStrategy)

- [Massi Tobo - Elado](https://www.youtube.com/watch?v=0fXcx7dv4CU&ab_channel=WaxOn)